In [69]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import glob
import os
import math
from os import listdir
import data_visualization as dv
import pickle
import scipy 
from scipy.stats import norm
from scipy import stats
from tqdm import tqdm 

In [70]:
# Fields pull and pull list
experiment1 = 1
if experiment1 == 1:
    os.chdir('D:\Subject_Data\Seth_MatchPennies_Agent_Pilot_v3')
PATH = os.getcwd()
analysis_pull_list = []
analysis_pull_list_react = []
analysis_pull_list_task = []
fields_pull = []
with open(PATH+"\\Analysis_Pull_List_React.txt", "r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
with open(PATH+"\\Analysis_Pull_List_React.txt", "r") as pull_file:
    analysis_pull_list_react = pull_file.read().splitlines()
with open(PATH+"\\Analysis_Pull_List_Task.txt", "r") as pull_file:
    analysis_pull_list_task = pull_file.read().splitlines()
with open(PATH+"\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()

# SPECIFIC TO EXPERIMENT
reaction_trials = 50
timing_trials = 50
num_trials = 100  # number of trials in each block
num_blocks = 6
tot_trials = (num_trials*num_blocks)
data_columns = len(fields_pull)
trial_time = 1500
num_subjects = len(analysis_pull_list_react)

# Need to be kept outside subject for-loop
trial_end = np.zeros([num_subjects, num_blocks, num_trials, 2])*np.nan
trial_start = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
filename_storage = np.empty((num_subjects, num_blocks, num_trials), dtype=object)
csv_files = np.empty((num_subjects, tot_trials), dtype=object)
csv_block_ordered = np.empty((num_subjects, num_blocks, num_trials))


player_decision_array = np.empty((num_subjects, num_blocks, num_trials))
player_reach_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_decision_time = np.zeros(
    [num_subjects, num_blocks, num_trials])*np.nan

hand_pos_data = np.zeros(
    (num_subjects, num_blocks, num_trials, trial_time+500, 4))*np.nan

event_codes = np.empty(
    (num_subjects, num_blocks, num_trials, trial_time+500), str)
list_event_codes = ["E_SOUND_SIGNAL", "E_END_REACHED_A", "E_END_REACHED_B"]
tp_list = ("TP4", "TP3", "TP2")
scanned_files = []

win_check_R = np.zeros((num_subjects, num_blocks))
win_check_L = np.zeros((num_subjects, num_blocks))
num_miss_check_R = np.zeros((num_subjects, num_blocks))
num_miss_check_L = np.zeros((num_subjects, num_blocks))



#########
agent_decision_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_target_selection = np.zeros((num_subjects, num_blocks, num_trials))




Target information

Row 0 = Start Target

Row 1 = Right screen, right target (all other target positions are based off of this)

Dim 1 = Radius

Dim 2 = Thickness of the circle edge (don't know if this matters)


Dataframe is in centimeters, so need to divide everything by 100

In [71]:
0.0075/2

0.00375

In [72]:
file = PATH+'\\Sub1_Task\\Sub1_TaskTarget_Table.csv'
df = pd.read_csv(file)
df["X"] = df["X"]/100
df["Y"] = df["Y"]/100
df['Dim 1'] = df['Dim 1']/100
df['Dim 2'] = df['Dim 2']/100
# Target information for Right Hand
start1x = df.loc[0]['X']
start1y = df.loc[0]['Y']
start1_radius = df.loc[0]['Dim 1'] 

target1x = df.loc[1]['X']
target1y = df.loc[1]['Y']
t1_pos = np.sqrt(target1x**2 + target1y**2)
target1_radius = df.loc[1]['Dim 1']

target2x = 2*start1x - target1x
target2y = target1y
t2_pos = np.sqrt(target2x**2 + target2y**2)
target2_radius = target1_radius

# for Left Hand
start2x = -0.55
start2y = 0.15
start2_radius = 0.015

target3x = target1x - 2*start1x
target3y = target1y
t3_pos = np.sqrt(target3x**2 + target3y**2)
target3_radius = target1_radius

target4x = 2*start2x - target3x
target4y = target1y
t4_pos = np.sqrt(target4x**2 + target4y**2)
target4_radius = target1_radius

# Timing target
timing_targetx = start2x
timing_targety = target1y
timing_target_pos = np.sqrt(timing_targetx**2 + timing_targety**2)
timing_target_radius = target1_radius   

In [73]:
df["Y"]

0    0.15000
1    0.31664
2    0.00000
3    0.00000
4    0.00000
5    0.32000
6    0.05000
7    0.22500
8    0.05500
9    0.15000
Name: Y, dtype: float64

In [74]:
df

,X,Y,First Color,Second Color,Dim 1,Dim 2,Dim 3,Dim 4,Dim 5,Third Color,True_X,True_Y
0,0.5500,0.15000,255255248,0,0.0075,0.0000,0,0.0,0,255255248,55.000000,15.000
1,0.6611,0.31664,0,255255248,0.0400,0.0025,0,0.0,0,0,66.110001,31.664
2,0.0000,0.00000,255255008,0,0.0050,0.0000,0,0.0,0,0,0.000000,0.000
3,0.0000,0.00000,255255008,0,0.0050,0.0000,0,0.0,0,0,0.000000,0.000
4,0.0000,0.00000,191255,0,0.0050,0.0000,0,0.0,0,0,0.000000,0.000
5,0.6000,0.32000,1000000000,0,0.0040,0.1000,6,0.5,4,0,60.000000,32.000
6,0.5500,0.05000,255255248,0,0.0000,0.2000,2,0.0,0,0,55.000000,5.000
7,0.5500,0.22500,0,255000000,0.0025,0.3500,30,0.0,0,0,55.000000,22.500
8,0.5000,0.05500,255000000,0,0.0006,0.0000,0,0.0,0,0,50.000000,5.500
9,-0.5500,0.15000,191255,0,0.0050,0.0000,0,0.0,0,0,-55.000000,15.000


In [75]:
#%% Get trial table for reaction and timing
reaction_timing_trial_table = np.empty((num_subjects, reaction_trials+timing_trials, 4), int)
task_trial_table = np.empty((num_subjects, tot_trials, 4))
for w in range(len(analysis_pull_list_react)): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_react[w]
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    df = pd.read_csv(file)

    df = df.loc[df['TP_Row']<3]
    reaction_timing_trial_table[w,:,:] = df[['Trial_Num','Block_Row','Block_Step','TP_Row']]

    
#%% Get trial table for task
for w in range(len(analysis_pull_list_task)): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_task[w]
    path1 = PATH+'\\'+sub_name
    df = pd.read_csv(path1+f'\\{sub_name}Trial_Table.csv')
    df = df.loc[df['TP_Row']>=3]
    task_trial_table[w,:,:] = df[['Trial_Num','Block_Row','Block_Step','TP_Row']]

0
1
2
3
4
0
1
2
3
4


In [76]:
df

,Trial_Num,Block_Row,Block_Step,TP_Row,Starting bar Targ No,Ending bar Targ No,Condition type,Condition time,Trial feedback,Biased Score Param,Biased score side,Biased score,Show cum scores,Spring constant wall,Damping constant wall,Score show time,Reward display,Agent Mean Reach Time,Agent SD Reach Time,Repeat_Trials
0,1,1,1,5,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1100,50,0
1,2,1,2,5,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1100,50,0
2,3,1,3,5,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1100,50,0
3,4,1,4,5,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1100,50,0
4,5,1,5,5,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1100,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,6,96,8,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1200,150,0
596,597,6,97,8,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1200,150,0
597,598,6,98,8,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1200,150,0
598,599,6,99,8,1,2,3,1500,2,0,0,0,0,6000,5,1000,0,1200,150,0


Reaction time data

In [77]:
#%% Get reaction time data
reaction_timing_trial_time = 2500
rt_decision_array = np.empty((num_subjects, reaction_trials))
rmt_decision_array = np.empty((num_subjects, reaction_trials))
rt_trial_start = np.zeros((num_subjects, reaction_trials))*np.nan
reaction_movement_time = np.zeros([num_subjects, reaction_trials])*np.nan
reaction_time = np.zeros([num_subjects, reaction_trials])*np.nan
rt_hand_pos_data = np.zeros((num_subjects, reaction_trials, reaction_timing_trial_time, 4))*np.nan

timing_reach_time = np.zeros([num_subjects, reaction_trials])*np.nan
for w in range(num_subjects):
    sub_name = analysis_pull_list_react[w]
    print(w, sub_name)
    block_number = 1
    tp_num = 1
    sub_name = analysis_pull_list_react[w]
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    reaction_trial_table = pd.read_csv(file)
    #reaction_trial_table = reaction_trial_table.loc[reaction_trial_table['TP_Row']==1]
    for i in tqdm((range(reaction_trials+timing_trials))):
        block_number = reaction_trial_table.iloc[i]['Block_Row']
        tp_num = reaction_trial_table.iloc[i]['TP_Row']
        block_trial_num = reaction_trial_table.iloc[i]['Block_Step']
        x = tp_num - 1
        c = block_trial_num - 1
        filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"

        data = pd.read_csv(filename, low_memory=False)
        rt_trial_start[w,c]= int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0]) #if event codes column is equal to the list event codes, give the index (time)
        rt_trial_start = rt_trial_start.astype(int)
        end_time = rt_trial_start[w,c]+reaction_timing_trial_time

        data = np.array(data)
        hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
        hand_data = hand_data.astype(float)
        rt_hand_pos_data[w,c,:,:] = hand_data[rt_trial_start[w,c]:end_time,:]
         # LEFT HAND
        lhx = rt_hand_pos_data[w,c,:,2]
        lhy = rt_hand_pos_data[w,c,:,3]
        if tp_num == 1:
            q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_radius)
            r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_radius) 
            s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_radius)
            if np.size(s)>0: 
                reaction_time[w,c] = s[0]
            if np.size(q)>0: #if LH enters right target
                rmt_decision_array[w,c] = 1
                reaction_movement_time[w,c] = q[0]
            elif np.size(r)>0: #if LH enters left target
                rmt_decision_array[w,c] = -1   
                reaction_movement_time[w,c] = r[0]
        # Used if doing timing trials
        if tp_num == 2:
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                timing_reach_time[w,c] = q[0]


0 Sub1_React


100%|██████████| 100/100 [00:01<00:00, 68.63it/s]


1 Sub2_React


100%|██████████| 100/100 [00:01<00:00, 61.18it/s]


2 Sub3_React


100%|██████████| 100/100 [00:01<00:00, 71.42it/s]


3 Sub4_React


100%|██████████| 100/100 [00:01<00:00, 68.30it/s]


4 Sub5_React


100%|██████████| 100/100 [00:01<00:00, 60.62it/s]


Trial Data 

In [78]:
#%% Extract Hand Position data
for w in range(num_subjects): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_task[w]
    print(sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    task_trial_table = pd.read_csv(file)
    task_trial_table = task_trial_table.loc[task_trial_table['TP_Row']>=3]
    for i in tqdm((range(tot_trials))):
        block_number = task_trial_table.iloc[i]['Block_Row']
        tp_num = task_trial_table.iloc[i]['TP_Row']
        block_trial_num = task_trial_table.iloc[i]['Block_Step']
        x = tp_num - 3 #used for indexing, subtraction number needs to change to 1 if I start with TP1 
        c = block_trial_num - 1 #used for indexing, block trial number starts with 1
        filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        filename_storage[w,x,c] = filename
        data = pd.read_csv(filename, low_memory=False) #read in data
        #Create start time and max end time array
        score_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
        trial_start[w,x,c]= int(data[data['Event_Codes']==list_event_codes[0]].index[0]) #if event codes column is equal to the list event codes, give the index (time)
        trial_start = trial_start.astype(int)
        end_time = trial_start[w,x,c]+2000 # 2000 to get reach times past 1500, constrained in if statement at bottom 
        agent_target_select_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
        # Get Agent mean reach time... data starts at next index from E_SOUNG_SIGNAL aka trial_start
        agent_decision_time[w,x,c] = data.iloc[trial_start[w,x,c]+1]['Agent_Reach_Time'] #CALLED AGENT REACH TIME BUT IT'S NOW DECISION TIME IN V3
        agent_target_selection[w,x,c] = data.iloc[agent_target_select_index]['Agent_Target_Selection']
        agent_indecision_index = np.argwhere(agent_decision_time+150>1500)
        for i,j,k in agent_indecision_index:
            agent_target_selection[i,j,k] = 0
        data = np.array(data)
        event_codes[w,x,c,:] = data[trial_start[w,x,c]:end_time,4] # create event codes array
        hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
        hand_data = hand_data.astype(float)
        hand_pos_data[w,x,c,:,:] = hand_data[trial_start[w,x,c]:end_time,:]
        
        # If you want the scores
        # scoreA = data[score_index, 4]
        # scoreB = data[score_index, 5]
        # if scoreA == 1:
        #     win_check_R[w,x]+=1
        # if scoreB == 1:
        #     win_check_L[w,x]+=1
        # print(filename)
        # print(data[trial_start[w,x,c]:end_time,:])
        #------------------------------------------------------------------
           
        # LEFT HAND
        lhx = hand_pos_data[w,x,c,:,2]
        lhy = hand_pos_data[w,x,c,:,3]
        q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_radius)
        r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_radius) 
        s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_radius)
        if np.size(s)>0: 
            player_decision_time[w,x,c] = s[0]
        if np.size(q)>0: #if LH enters right target
            player_decision_array[w,x,c] = 1
            player_reach_time[w,x,c] = q[0]
        elif np.size(r)>0: #if LH enters left target
            player_decision_array[w,x,c] = -1   
            player_reach_time[w,x,c] = r[0]
            
        if player_reach_time[w,x,c] > 1500 or np.isnan(player_reach_time[w,x,c]):
            player_decision_array[w,x,c] = 0
            
        # Plot hand paths on indecisions
        # if tp_num == 3:
        #     if player_decision_array[w,x,c] == 0:   
        #         lhx_new = hand_pos_data[w,x,c,:1500,2]
        #         lhy_new = hand_pos_data[w,x,c,:1500,3]
        #         #plot path for TargetNotReached
        #         #plt.figure(dpi=300)
        #         circleR = plt.Circle((target3x,target3y), target3_radius, color = 'r', fill = False)
        #         circleL = plt.Circle((target4x,target4y), target4_radius, color = 'r', fill = False)
        #         fig, ax = plt.subplots()
        #         ax.add_patch(circleR)
        #         ax.add_patch(circleL)
        #         plt.plot(lhx_new,lhy_new) 

        #         plt.title("w =%1.0f " %w + "x=%1.0f "%x + "c=%1.0f"%c)
        #         plt.show()

0
Sub1_Task


100%|██████████| 600/600 [00:28<00:00, 20.69it/s]


1
Sub2_Task


100%|██████████| 600/600 [00:34<00:00, 17.30it/s]


2
Sub3_Task


100%|██████████| 600/600 [00:19<00:00, 30.47it/s]


3
Sub4_Task


100%|██████████| 600/600 [00:27<00:00, 21.57it/s]


4
Sub5_Task


100%|██████████| 600/600 [00:28<00:00, 21.30it/s]


Use event codes to get wins and indecisions

In [79]:
# # Use event codes to find number of wins and indecisions
# #%% Extract Hand Position data
# for w in range(num_subjects): #len(pull_list)):
#     print(w)
#     sub_name = analysis_pull_list_task[w]
#     print(sub_name)
#     path1 = PATH+'\\'+sub_name
#     file = path1+f'\\{sub_name}Trial_Table.csv'
#     task_trial_table = pd.read_csv(file)
#     task_trial_table = task_trial_table.loc[task_trial_table['TP_Row']>=3]
#     for i in tqdm((range(tot_trials))):
#         block_number = task_trial_table.iloc[i]['Block_Row']
#         tp_num = task_trial_table.iloc[i]['TP_Row']
#         block_trial_num = task_trial_table.iloc[i]['Block_Step']
#         x = tp_num - 3 #used for indexing, subtraction number needs to change to 1 if I start with TP1 
#         c = block_trial_num - 1 #used for indexing, block trial number starts with 1
#         filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
#         filename_storage[w,x,c] = filename
#         data = pd.read_csv(filename, low_memory=False) #read in data
#         #Create start time and max end time array
#         score_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         trial_start[w,x,c]= int(data[data['Event_Codes']==list_event_codes[0]].index[0]) #if event codes column is equal to the list event codes, give the index (time)
#         trial_start = trial_start.astype(int)
#         end_time = trial_start[w,x,c]+2000
#         agent_target_select_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         # Get Agent mean reach time... data starts at next index from E_SOUNG_SIGNAL aka trial_start
#         agent_decision_time[w,x,c] = data.iloc[trial_start[w,x,c]+1]['Agent_Reach_Time'] #CALLED AGENT REACH TIME BUT IT'S NOW DECISION TIME IN V3
#         agent_target_selection[w,x,c] = data.iloc[agent_target_select_index]['Agent_Target_Selection']
#         agent_indecision_index = np.argwhere(agent_decision_time+150>1500)
#         for i,j,k in agent_indecision_index:
#             agent_target_selection[i,j,k] = 0
#         data = np.array(data)
#         event_codes[w,x,c,:] = data[trial_start[w,x,c]:end_time,4] # create event codes array
#         hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
#         hand_data = hand_data.astype(float)
#         hand_pos_data[w,x,c,:,:] = hand_data[trial_start[w,x,c]:end_time,:]
        
#         # If you want the scores
#         # scoreA = data[score_index, 4]
#         # scoreB = data[score_index, 5]
#         # if scoreA == 1:
#         #     win_check_R[w,x]+=1
#         # if scoreB == 1:
#         #     win_check_L[w,x]+=1
#         # print(filename)
#         # print(data[trial_start[w,x,c]:end_time,:])
#         #------------------------------------------------------------------
           
#         # LEFT HAND
#         lhx = hand_pos_data[w,x,c,:,2]
#         lhy = hand_pos_data[w,x,c,:,3]
#         q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_radius/2)
#         r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_radius/2) 
#         s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_radius/2)
#         if np.size(s)>0: 
#             player_decision_time[w,x,c] = s[0]
#         if np.size(q)>0: #if LH enters right target
#             player_decision_array[w,x,c] = 1
#             player_reach_time[w,x,c] = q[0]
#         elif np.size(r)>0: #if LH enters left target
#             player_decision_array[w,x,c] = -1   
#             player_reach_time[w,x,c] = r[0]
            
#         if player_reach_time[w,x,c] > 1500 or np.isnan(player_reach_time[w,x,c]):
#             player_decision_array[w,x,c] = 0

In [80]:
# Win, Loss, Indecision counts
# Make sure agent_target_selection replaces 2 with -1 from the first pilot
agent_target_selection = np.where(agent_target_selection ==2,-1,agent_target_selection)
player_indecisions = np.zeros((num_subjects, num_blocks))
player_wins = np.zeros((num_subjects, num_blocks))
player_incorrect_decisions = np.zeros((num_subjects,num_blocks))
for i in range(num_subjects):
    for j in range(num_blocks):
        player_indecisions[i,j] = np.count_nonzero(player_decision_array[i,j,:] == 0)
        player_wins[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == 1))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == -1))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == 0))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == 0))
        player_incorrect_decisions[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == -1))
        player_incorrect_decisions[i,j] += np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == 1))
perc_player_indecisions = (player_indecisions / num_trials)*100
perc_player_wins = (player_wins / num_trials)*100
perc_player_incorrect_decisions = (player_incorrect_decisions/num_trials)*100

In [81]:
for i in range(num_subjects):
    for j in range(num_blocks):
        print(player_indecisions[i,j],player_wins[i,j],player_incorrect_decisions[i,j])

5.0 87.0 8.0
13.0 83.0 4.0
6.0 61.0 33.0
49.0 49.0 2.0
35.0 53.0 12.0
11.0 72.0 17.0
12.0 88.0 0.0
8.0 67.0 25.0
12.0 51.0 37.0
6.0 76.0 18.0
26.0 59.0 15.0
11.0 64.0 25.0
11.0 65.0 24.0
8.0 64.0 28.0
23.0 43.0 34.0
7.0 65.0 28.0
1.0 72.0 27.0
13.0 67.0 20.0
4.0 66.0 30.0
5.0 83.0 12.0
9.0 53.0 38.0
7.0 66.0 27.0
14.0 56.0 30.0
20.0 66.0 14.0
1.0 99.0 0.0
20.0 77.0 3.0
37.0 62.0 1.0
40.0 60.0 0.0
66.0 30.0 4.0
30.0 64.0 6.0


In [82]:
# Calculate last 80 trials
n = 20 
player_indecisions_cut_off = np.zeros((num_subjects, num_blocks))
player_wins_cut_off = np.zeros((num_subjects, num_blocks))
player_incorrect_decisions_cut_off = np.zeros((num_subjects,num_blocks))
for i in range(num_subjects):
    for j in range(num_blocks):
        player_indecisions_cut_off[i,j] = np.count_nonzero(player_decision_array[i,j,n:] == 0)
        player_wins_cut_off[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == 1))
        player_wins_cut_off[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == -1))
        player_wins_cut_off[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == 0))
        player_wins_cut_off[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == 0))
        player_incorrect_decisions_cut_off[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == -1))
        player_incorrect_decisions_cut_off[i,j] += np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == 1))
        player_indecisions_cut_off[i,j] = np.count_nonzero(player_decision_array[i,j,n:] == 0)
perc_player_indecisions_cut_off = (player_indecisions_cut_off / (num_trials-n))*100
perc_player_wins_cut_off = (player_wins_cut_off / (num_trials-n))*100
perc_player_incorrect_decisions_cut_off = (player_incorrect_decisions_cut_off / (num_trials-n))*100

In [83]:
# Calculate first half
n=50
player_indecisions_first_half = np.zeros((num_subjects, num_blocks))
player_wins_first_half = np.zeros((num_subjects, num_blocks))
player_incorrect_decisions_first_half = np.zeros((num_subjects,num_blocks))
for i in range(num_subjects):
    for j in range(num_blocks):
        player_indecisions_first_half[i,j] = np.count_nonzero(player_decision_array[i,j,:n] == 0)
        player_wins_first_half[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == 1 , agent_target_selection[i,j,:n] == 1))
        player_wins_first_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == -1 , agent_target_selection[i,j,:n] == -1))
        player_wins_first_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == -1 , agent_target_selection[i,j,:n] == 0))
        player_wins_first_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == 1 , agent_target_selection[i,j,:n] == 0))
        player_incorrect_decisions_first_half[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == 1 , agent_target_selection[i,j,:n] == -1))
        player_incorrect_decisions_first_half[i,j] += np.count_nonzero(np.logical_and(player_decision_array[i,j,:n] == -1 , agent_target_selection[i,j,:n] == 1))
        player_indecisions_first_half[i,j] = np.count_nonzero(player_decision_array[i,j,:n] == 0)
perc_player_indecisions_first_half = (player_indecisions_first_half / (num_trials-n))*100
perc_player_wins_first_half = (player_wins_first_half / (num_trials-n))*100
perc_player_incorrect_decisions_first_half = (player_incorrect_decisions_first_half / (num_trials-n))*100

In [84]:
# Calculate second half
n=50
player_indecisions_second_half = np.zeros((num_subjects, num_blocks))
player_wins_second_half = np.zeros((num_subjects, num_blocks))
player_incorrect_decisions_second_half = np.zeros((num_subjects,num_blocks))
for i in range(num_subjects):
    for j in range(num_blocks):
        player_indecisions_second_half[i,j] = np.count_nonzero(player_decision_array[i,j,n:] == 0)
        player_wins_second_half[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == 1))
        player_wins_second_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == -1))
        player_wins_second_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == 0))
        player_wins_second_half[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == 0))
        player_incorrect_decisions_second_half[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == 1 , agent_target_selection[i,j,n:] == -1))
        player_incorrect_decisions_second_half[i,j] += np.count_nonzero(np.logical_and(player_decision_array[i,j,n:] == -1 , agent_target_selection[i,j,n:] == 1))
        player_indecisions_second_half[i,j] = np.count_nonzero(player_decision_array[i,j,n:] == 0)
perc_player_indecisions_second_half = (player_indecisions_second_half / (num_trials-n))*100
perc_player_wins_second_half = (player_wins_second_half / (num_trials-n))*100
perc_player_incorrect_decisions_second_half = (player_incorrect_decisions_second_half / (num_trials-n))*100

In [85]:
# Pickle
analysis_pull_list = []
with open(PATH+"\\Analysis_Pull_List.txt","r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
i=0
for subname in analysis_pull_list:
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    # Reaction time 
    pickle.dump(rt_trial_start[i,:], open(data_path + f'{subname}_rt_trial_start.pkl', 'wb'))
    pickle.dump(reaction_movement_time[i,:], open(data_path + f'{subname}_reaction_movement_time.pkl', 'wb'))
    pickle.dump(reaction_time[i,:], open(data_path + f'{subname}_reaction_time.pkl', 'wb'))
    pickle.dump(rmt_decision_array[i,:], open(data_path + f'{subname}_rmt_decision_array.pkl', 'wb'))
    # Timing
    pickle.dump(timing_reach_time[i,:], open(data_path + f'{subname}_timing_reach_time.pkl', 'wb'))
    # Hand position thangs
    pickle.dump(player_indecisions[i,:],open(data_path + f'{subname}_player_indecisions.pkl', 'wb'))    
    pickle.dump(player_indecisions_first_half[i,:],open(data_path + f'{subname}_player_indecisions_first_half.pkl', 'wb')) #
    pickle.dump(player_indecisions_second_half[i,:],open(data_path + f'{subname}_player_indecisions_second_half.pkl', 'wb')) #
    pickle.dump(player_indecisions_cut_off[i,:],open(data_path + f'{subname}_player_indecisions_cut_off.pkl', 'wb'))    

    pickle.dump(player_wins[i,:],open(data_path + f'{subname}_player_wins.pkl', 'wb'))
    pickle.dump(player_wins_first_half[i,:],open(data_path + f'{subname}_player_wins_first_half.pkl', 'wb'))
    pickle.dump(player_wins_second_half[i,:],open(data_path + f'{subname}_player_wins_second_half.pkl', 'wb'))
    pickle.dump(player_wins_cut_off[i,:],open(data_path + f'{subname}_player_wins_cut_off.pkl', 'wb'))

    pickle.dump(perc_player_indecisions[i,:],open(data_path + f'{subname}_perc_player_indecisions.pkl', 'wb')) #
    pickle.dump(perc_player_indecisions_first_half[i,:],open(data_path + f'{subname}_perc_player_indecisions_first_half.pkl', 'wb')) #
    pickle.dump(perc_player_indecisions_second_half[i,:],open(data_path + f'{subname}_perc_player_indecisions_second_half.pkl', 'wb')) #
    pickle.dump(perc_player_indecisions_cut_off[i,:],open(data_path + f'{subname}_perc_player_indecisions_cut_off.pkl', 'wb')) #

    pickle.dump(perc_player_wins[i,:],open(data_path + f'{subname}_perc_player_wins.pkl', 'wb'))
    pickle.dump(perc_player_wins_first_half[i,:],open(data_path + f'{subname}_perc_player_wins_first_half.pkl', 'wb'))
    pickle.dump(perc_player_wins_second_half[i,:],open(data_path + f'{subname}_perc_player_wins_second_half.pkl', 'wb'))
    pickle.dump(perc_player_wins_cut_off[i,:],open(data_path + f'{subname}_perc_player_wins_cut_off.pkl', 'wb'))

    pickle.dump(player_incorrect_decisions[i,:], open(data_path + f'{subname}_player_incorrect_decisions.pkl', 'wb'))
    pickle.dump(perc_player_incorrect_decisions[i,:], open(data_path + f'{subname}_perc_player_incorrect_decisions.pkl', 'wb'))
    pickle.dump(perc_player_incorrect_decisions_first_half[i,:], open(data_path + f'{subname}_perc_player_incorrect_decisions_first_half.pkl', 'wb'))
    pickle.dump(perc_player_incorrect_decisions_second_half[i,:], open(data_path + f'{subname}_perc_player_incorrect_decisions_second_half.pkl', 'wb'))
    pickle.dump(perc_player_incorrect_decisions_cut_off[i,:], open(data_path + f'{subname}_perc_player_incorrect_decisions_cut_off.pkl', 'wb'))


    pickle.dump(hand_pos_data[i,:,:,:,:], open(data_path + f'{subname}_hand_pos_data.pkl', 'wb'))
    pickle.dump(trial_start[i,:,:], open(data_path + f'{subname}_trial_start.pkl', 'wb'))
    pickle.dump(player_reach_time[i,:,:], open(data_path + f'{subname}_player_reach_time.pkl', 'wb'))   
    #pickle.dump(trial_table[i,:,:], open(data_path + f'{subname}_trial_table.pkl', 'wb'))
    pickle.dump(player_decision_array[i,:,:], open(data_path + f'{subname}_player_decision_array.pkl', 'wb'))

    pickle.dump(player_decision_time[i,:,:], open(data_path + f'{subname}_player_decision_time.pkl', 'wb'))
    pickle.dump(player_decision_array[i,:,:], open(data_path + f'{subname}_player_decision_array.pkl', 'wb'))
    
    # Agent stuff
    pickle.dump(agent_target_selection[i,:,:],open(data_path + f'{subname}_agent_target_selection.pkl', 'wb'))
    pickle.dump(agent_decision_time[i,:,:],open(data_path + f'{subname}_agent_decision_time.pkl', 'wb'))

    i += 1         

Sub1
Sub2
Sub3
Sub4
Sub5


In [86]:
player_wins

array([[87., 83., 61., 49., 53., 72.],
       [88., 67., 51., 76., 59., 64.],
       [65., 64., 43., 65., 72., 67.],
       [66., 83., 53., 66., 56., 66.],
       [99., 77., 62., 60., 30., 64.]])